# Introduction to physics informed neural networks (PINNS)
In the previous examples we have trained neural networks to represent functions by feeding it with a set of observations (samples) at certain input points $x, y(x)$, and we have minimized the mean squared error (MSE) between observations and predictions.
This may require many training samples, particularly in the case of complex functions. However, if we have prior information regarding the physical behaviour of $y$ such information could be used in combination with observations when training a neural network. In this setting recent developments in [automatic differentiation](https://arxiv.org/abs/1502.05767) can be exploited, which allow to differentiate neural networks with respect to their input coordinates to obtain [physics informed neural networks](https://arxiv.org/abs/1711.10561) (PINNS). In this setting let us consider a general nonlinear partial differential equation of the general form

\begin{equation}
u_t +  \Lambda \left[u;\lambda \right] = 0\,,
\end{equation}

where $\Lambda$ is a differential operator, and $\lambda$ are parameters. Like for regular neural networks we would like to minimize discenpencies between observations $u\left(x_i, t^n \right)$ and predictions, however we would also like that our governing equation is satisfied, and hence want to minimize the left hand side of the govering equation:
\begin{equation}
f := u_t +  \Lambda \left[ u \right] \,,
\end{equation}

In order to obtain a loss function
\begin{equation}
loss  = \mathrm{MSE}_u + \mathrm{MSE}_f = \frac{1}{N_u}\sum_{i=1}^{N_u}\left(u\left(t_u^i, x_u^i\right) - u_i\right)^2 + \frac{1}{N_f}\sum_{i=1}^{N_f}\left(f\left(t_f^i, x_f^i\right)\right)^2  \,,
\end{equation}

i.e. by evaluting the loss function based on the neural nets performance for predicting $u$, a set of $\left\{t_u^i, x_u^i, u^i\right\}_{i=1}^{N_u}$ training points (initial and boundary data) and by additionally evaluating the left hand side of the governing equation, $f$ on a set of points $\left\{t_f^i, x_f^i\right\}_{i=1}^{N_f}$.

## Solve ODEs by minimizing a combination of observations and physical (equations) 

In order to introuduce PINNS we'll start by training a neural net to represent the exponential function $y\left(x\right) = e^x$. We'll start by learning to represent the function from a set of (training) points, y. We'll then see how we can train the same network by constraining it to satisfy (or at least minimize deviation from) it's governing equation 

\begin{equation}
\frac{d \,y\left(x\right)}{dx}  - y\left(x\right) = 0  \,.
\end{equation}

 In this case we have only one independent variable, $x$, and the loss function for the regual neural network, and for the PINN simplify to:


\begin{equation}
\mathrm{loss}_\mathrm{NN} =  \mathrm{MSE}_y  = \frac{1}{N_y}\sum_{i=1}^{N_y}\left(u\left(x_y^i\right) - y_i\right)^2  \,,
\end{equation}

\begin{equation}
\mathrm{loss}_\mathrm{PINN}  = \mathrm{MSE}_u + \mathrm{MSE}_f = \frac{1}{N_{y, PINN}}\sum_{i=1}^{N_{y, PINN}}\left(u\left(x_y^i\right) - y_i\right)^2 + \frac{1}{N_f}\sum_{i=1}^{N_f}\left(f\left(x_f^i\right)\right)^2  \,,
\end{equation}

where $f = y_x - y$. Note that the number of observations $N_y$ and $N_{y, PINN}$ does not have to be the same in $\mathrm{loss}_\mathrm{NN}$ and $\mathrm{loss}_\mathrm{PINN}$, since in general we would like to use PINNS in cases where we have sparse amount of data/observations. In fact (in this example) we will only provide the PINN with one observed $y$ value. The number of points $N_f$ where $f$ is evaluated, on the other hand, is not constrained by lack of data, and we are in principle free to chose as many points as we want. But in this example we'll compare the regular NN  and PINN in cases where $N_{y, PINN} + N_f \approx N_y$.

In both cases we'll use a single hidden layer, like the one below, however the number of neurons can be changed.


<img src="fig/neuralNet.png" width="400">

## Problems
- Compare accuracy (MSE) using the regular NN and PINN
- Compare accuracy (MSE) using the regular NN and PINN in the case of extrapolation, i.e. when training is done on a limited set of data, and predictions are obtained outside this area.
- Modify the code to represent $y(x)$ = cos(x), in which $f = y + \frac{d^2\,y}{dx^2}$
- Modify the code to represent y(x) = e^x * cos(x), in which $f: y - \frac{dy}{dx} + 0.5\,\frac{d^2 \, y}{dx^2}$
- In the latter two cases, try to increase the domain bounds. Also try with and without feeding the NN with scaled (normalized between 0 and 1) inputs. For the PINN you may need to feed it with a few more observations.

## Regular neural network

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#==========================================================================================#
# Set parampeters (hyperameters) and compute a training data set for the desired function  #
#==========================================================================================#

N = 10001 # Number of training points
N_train = 10 # Number of points to train on per training attempt
N_neurons = 5 # Number of neurons in the hidden layer
afunc = tf.nn.sigmoid # Set activation function, other functions .relu, .sigmoid, .tanh, .elu

learning_rate = 0.1
epochs = 5000 # Number of epochs

#Set x-domain boundaries
x_start = 0.
x_end = 2.

#ub = np.array([[x_end]]) #upper bound
x_data = np.linspace(x_start, x_end, N)
#===================================================
# Specify what function you wish to train for
#===================================================
y_data = np.exp(x_data)


In [ ]:
x_data = x_data[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)
y_data = y_data[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)

constrainUpper = 1. # set smaller than one to test extrapolation
idx = np.random.choice(int(x_data.shape[0]*constrainUpper), N_train, replace=False)
x_data_train = x_data[idx]
y_data_train = y_data[idx]

In [ ]:
#===================================================
# set up placeholder for inputs and outputs
#===================================================
x = tf.placeholder(tf.float32, shape=[None, x_data.shape[1]], name='x')
y = tf.placeholder(tf.float32, shape=[None, y_data.shape[1]], name='y')
#===================================================
# declare weights and biases input --> hidden layer
#===================================================
W1 = tf.Variable(tf.random_normal([1, N_neurons], stddev=0.5), name='W1')
b1 = tf.Variable(tf.random_normal([N_neurons]), name='b1')
#===================================================
# declare weights and biases of hidden --> output layer
#===================================================
W2 = tf.Variable(tf.random_normal([N_neurons, 1], stddev=0.5), name='W2')
b2 = tf.Variable(tf.random_normal([1]), name='b2')
#===================================================
# declare output of NN
#===================================================
a1 = afunc(tf.add(tf.matmul(x, W1), b1)) # activation of hidden layer
y_NN = tf.add(tf.matmul(a1, W2), b2) # computational graph for the neural network

In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    y_pred_init = sess.run(y_NN, feed_dict = {x:x_data})
plt.figure()
plt.plot(x_data.flatten(), y_data.flatten(), label="Solution")
plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'o', label = 'Training points')
plt.plot(x_data.flatten(), y_pred_init.flatten(), label='Initialization values')
plt.title('Initial state and training points')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
#===================================================
# train the model using regular NN
#===================================================
loss = tf.reduce_mean(tf.square(y - y_NN))
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser, loss], 
                     feed_dict={x: x_data_train, y: y_data_train})
        avg_cost += c
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost))

    y_pred = sess.run(y_NN, feed_dict = {x:x_data, y:y_data})
    loss_pred = sess.run(loss, feed_dict = {x:x_data, y:y_data})
    plt.figure()
    plt.plot(x_data.flatten(), y_data.flatten(), 'o')
    plt.plot(x_data.flatten(), y_pred.flatten(),'r--')
    plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'bo')
    plt.title('Regular Neural Network Solution')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(["y", "y_NN", "y_train"])
    plt.show()
    print("MSE_u on all data (N: {}): ".format(x_data.shape[0]), loss_pred)

## PINN

In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    y_pred_init = sess.run(y_NN, feed_dict = {x:x_data})
plt.figure()
plt.plot(x_data.flatten(), y_data.flatten(), label='Solution')
plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'o', label='Training points')
plt.plot(x_data.flatten(), y_pred_init.flatten(), label='Initialization values')
plt.title('Initial state and training points')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
#===================================================
# train the model using PINNS
#===================================================

# Define placeholders for the neural net estimating the differential equation
x_f = tf.placeholder(tf.float32, shape=[None, x_data.shape[1]], name='x_f')
a1_PINNS = afunc(tf.add(tf.matmul(x_f, W1), b1))
y_NN_PINNS = tf.add(tf.matmul(a1_PINNS, W2), b2)
# note that dy_dx_NN, y_NN_PINNS and y_NN share the same weights and biases
dy_dx_NN = tf.gradients(y_NN_PINNS, x_f)[0]

f =  dy_dx_NN - y_NN_PINNS

# Define loss function including the differential equation term
MSE_u = tf.reduce_mean(tf.square(y - y_NN))
MSE_f = tf.reduce_mean(tf.square(f))
loss_PINNS = MSE_u + MSE_f
optimiser_PINNS = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_PINNS)

# We only use one training point (the first) to minimize MSEu
x_data_train_PINNS = x_data_train[0:1,0:1] 
y_data_train_PINNS = y_data_train[0:1,0:1]

# Run training of the physially informed neural net
print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser_PINNS, loss_PINNS ], 
                     feed_dict={x: x_data_train_PINNS, y: y_data_train_PINNS, x_f: x_data_train})
        avg_cost += c
        MSE_u_value, MSE_f_value = sess.run([MSE_u, MSE_f], 
                     feed_dict={x: x_data_train_PINNS, y: y_data_train_PINNS, x_f: x_data_train})
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost), "MSE_u =", "{:.6f}".format(MSE_u_value), "MSE_f =", "{:.6f}".format(MSE_f_value))

    y_pred_PINN = sess.run(y_NN, feed_dict = {x:x_data, y:y_data})
    loss_pred = sess.run(loss, feed_dict = {x:x_data, y:y_data, x_f:x_data})
    plt.figure()
    plt.plot(x_data.flatten(), y_data.flatten(), 'o')
    plt.plot(x_data.flatten(), y_pred_PINN.flatten(),'r--')
    plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'bo')
    plt.plot(x_data_train_PINNS.flatten(), y_data_train_PINNS.flatten(), 'go')
    plt.legend(["y", "y_NN", "y_train(x_f)", "y_train"])
    plt.title('Physically Informed Neural Network Solution')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    print("MSE_u on all data (N: {}): ".format(x_data.shape[0]), loss_pred)


In [ ]:
#===================================================
# plot solution from regular and PINN 
#===================================================
plt.figure()
plt.plot(x_data.flatten(), y_data.flatten(), 'o', markerfacecolor="None")
plt.plot(x_data.flatten(), y_pred_PINN.flatten(),'r--')
plt.plot(x_data.flatten(), y_pred.flatten(),'k--')
plt.legend(["y", "y_NN", "y_NN_PINN"]) 
plt.title('Solution comparison')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Sympy cells to calculate f for some simple functions

In [ ]:
import sympy as sp

x_sp = sp.Symbol('x')
y_sp = sp.cos(x_sp)
dy_sp_dx = sp.diff(y_sp, x_sp)
dy_sp_dx2 = sp.diff(dy_sp_dx, x_sp)
#y_sp, dy_sp_dx, dy_sp_dx2
y_sp + dy_sp_dx2

In [ ]:
x_sp = sp.Symbol('x')
y_sp = sp.cos(x_sp)*sp.exp(x_sp)
dy_sp_dx = sp.diff(y_sp, x_sp)
dy_sp_dx2 = sp.diff(dy_sp_dx, x_sp)
#y_sp, dy_sp_dx, dy_sp_dx2
y_sp - dy_sp_dx + 0.5*dy_sp_dx2

# PINN - The Two Element Windkessel model


Here we will implement the two element Windkessel model, which we saw in chapter 1.2, in a Pysically Informed Neural Network. The model is characterized by the parameters resistance $R$ and compliance $C$ as before. The equation describing the model is the following:

\begin{equation*}
\frac{\mathrm{d}p}{\mathrm{dt}} = \frac{Q_{\mathrm{in}}}{C} - \frac{p}{R C}, 
\end{equation*}

where $Q_{\mathrm{in}}$ is the flow entering the compliant volume at a given point in time. If we use this model to produce data and then build a loss function implementing the equation above, taking the input flow and a few points sampled from the model produced pressure function we can train a PINN to appoximate the model. 

In the code below, $f$ refers to the sum:

\begin{equation*}
f = \frac{\mathrm{d}p}{\mathrm{dt}} - \frac{Q_{\mathrm{in}}}{C} + \frac{p}{R C}
\end{equation*}

which ideally should be equal to zero, and must be minimized to obey the physics of the problem. $u$ refers to our variable of interest, which here is $p$ used interchangeably with $P_{ao}$. 

Problem 2:
- Tune the hyperparameters; number of nodes, layers, epochs, training rate, but also the number of pressure training points, number of flow training points, and try to get a good fit. Can you implement an ordinary neural network which can learn the pressure? Which can you get to perform best?

For this problem a gaussian inflow is prescribed.

## Code and functions to calculate solution and flow for given model parameters

Note: Rerun the entire code from this point on if you run any code in the section on 1D blood flow and then go back here.

In [ ]:
import numpy as np
import scipy.integrate as scint
import matplotlib.pyplot as plt
%matplotlib inline

#============================================================#
# Set up a 2 element Windkessel model for production of data #
#============================================================#

class WK2():
    def __init__(self, pars):
        self.R_sys = pars["R_sys"]
        self.C_sa = pars["C_sa"]
    
    def gaussian_flow(self,t):
        gaussian = 450.0*np.exp(-(np.mod(t,1.0)-0.2)**2 / (0.09)**2)
        return gaussian
    
    def rhs(self, t, u):
        P_ao = u[0]
        Q_lvao = self.gaussian_flow(np.mod(t,1.0))
        Q_aosv = P_ao/self.R_sys
        
        der_P_ao = (Q_lvao - Q_aosv)/self.C_sa
        der_u = [der_P_ao]
        
        return der_u

    def calc_all(self, t, u):
        P_ao = u[0]
        Q_lvao = self.gaussian_flow(np.mod(t,1.0))
        Q_aosv =  P_ao/self.R_sys
        
        all_vars = locals()
        del all_vars["self"]  
        del all_vars["u"]

        der_P_ao = (Q_lvao - Q_aosv)/self.C_sa
        der_u = [der_P_ao]

        return der_u, all_vars
    
def get_data(N, showPlots=True):
    #Set initial condition
    u0 = [100.0]
    
    #Set up a model example
    Parameters = dict(R_sys=1.5,C_sa=1.5)
    wkmod = WK2(Parameters)
    Ncycles = 20
    tmax = Ncycles*1.0 #Ncycles
    t_eval_vals = np.linspace(0.0, tmax, N*Ncycles)
    
    
    sol = scint.solve_ivp(wkmod.rhs, (0.0, tmax), u0, max_step=0.01, t_eval=t_eval_vals)
    _, all_vars = wkmod.calc_all(sol.t, sol.y)
    P_ao_fin_cyc = all_vars["P_ao"][int((Ncycles-1)*N):]
    t_fin = all_vars["t"][0:N]
    q_lvao = wkmod.gaussian_flow(t_fin)
    
    #P_np, P_q_np, integrand_q_array = calcDeltaP(t_in, integrand_q, P_in=0)
    if showPlots:
        plt.figure()
        plt.plot(t_fin, P_ao_fin_cyc)
        plt.legend(["P_ao"])
        #plt.plot(x_np, linearTapering_np(x_np, R0, Rmin, l))
        plt.xlabel("t [s]")
        plt.ylabel("P [mmHg]")
        plt.title("Pressure curve for data samples")
        plt.show()
    
    return t_fin, P_ao_fin_cyc, q_lvao, wkmod.R_sys, wkmod.C_sa

## Code and functions do define neural net, f-function and initialization of variables

In [ ]:
#=============================================#
# Functions for setting up the neural network #
#=============================================#

def net_u(t):
    u = neural_net(t, weights, biases)
    return u
#===================================================

def net_f_WK(t, Q, R, C):
    
    u = net_u(t)
    
    u_t = tf.gradients(u, t)
    
    f =  u_t + u/(R*C) - Q/C
    
    return f
#===================================================

def initialize_NN(layers):        
    weights = []
    biases = []
    num_layers = len(layers) 
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l+1]])
        #W = tf.Variable(tf.random_normal([layers[l], layers[l+1]], stddev=10))
        b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        weights.append(W)
        biases.append(b)        
    return weights, biases
#===================================================
  
def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(2/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
#===================================================

def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    
    H = 2.0*(X - lb)/(ub - lb) - 1.0
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.tanh(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y
#===================================================

def callback(loss):
    print('Loss:', loss)

## Main program, setting training parameters and NN layers

In [ ]:
#===================================================
# load data
#===================================================
N = 1001 # Total number of t, P values to generate
N_train = 1 # number of training points for NN
N_train_f = 50 # number of training points for f, i.e., input flow term and pressure term

#maxP = np.max(P_data)
t, P_data, Q_data, R, C = get_data(N, showPlots=False)
#P_data = P_data/maxP
#===================================================
# set parameters for neural network and training
#===================================================
layers = [1, 7, 1]

init = tf.global_variables_initializer()
#===================================================
# turn 1D array into 2D array of shape (N, 1)
#===================================================
t = t[:, np.newaxis]
P_data = P_data[:, np.newaxis]
Q_data = Q_data[:, np.newaxis]
#===================================================
# sample random points for training
#===================================================
idx = np.random.choice(t.shape[0], N_train, replace=False)
idx_f = np.random.choice(t.shape[0], N_train_f, replace=False)
t_train = t[idx]
P_train = P_data[idx]
#t_train = t[0:1,0:1]
#P_train = P[0:1,0:1]
t_f_train = t[idx_f]
q_train = Q_data[idx_f]
P_f_train = P_data[idx_f]

#===================================================
# Set up placeholder for inputs and outputs
#===================================================
C_tf = tf.constant(C)
R_tf = tf.constant(R)

t_tf = tf.placeholder(tf.float32, shape=[None, t.shape[1]], name='t')
P_tf = tf.placeholder(tf.float32, shape=[None, P_data.shape[1]], name='P')

t_f_tf = tf.placeholder(tf.float32, shape=[None, t.shape[1]], name='t')
q_tf = tf.placeholder(tf.float32, shape=[None, Q_data.shape[1]], name='Q')

#===================================================
# initialize neural net and f functions
#===================================================
lb = t.min(0)
ub = t.max(0)
weights, biases = initialize_NN(layers)

P_pred = net_u(t_tf)
f_pred = net_f_WK(t_f_tf, q_tf, C_tf, R_tf)


In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    P_pred_init = sess.run(P_pred, feed_dict = {t_tf:t})
    plt.figure()
    plt.plot(t.flatten(), P_data.flatten(), label = "Data")
    plt.plot(t_f_train.flatten(), P_f_train.flatten(), 'o', label="Diff. eq. training points")
    plt.plot(t_train.flatten(), P_train.flatten(), 'o', label="NN Training point")
    plt.plot(t.flatten(), P_pred_init.flatten(), label = "Initialization values")
    plt.title('Initialization values')
    plt.xlabel('t [s]')
    plt.ylabel('P [mmHg]')
    plt.legend()
    plt.show()

In [ ]:
#===========================================================#
# train the model using PINNS and GradientDescentOptimizer  #
#===========================================================#
batch_size = N_train
loss = tf.reduce_mean(tf.square(P_tf - P_pred)) + tf.reduce_mean(tf.square(f_pred))
learning_rate = 0.00012
#learning_rate_late = 0.5
epochs = 1000
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser, loss], 
                     feed_dict={t_tf: t_train, P_tf: P_train, 
                                t_f_tf: t_f_train, q_tf:q_train})
        avg_cost += c
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost))

    P_result = sess.run(P_pred, feed_dict = {t_tf:t})
    plt.figure()
    plt.plot(t.flatten(), P_data.flatten(), label='Data')
    plt.plot(t_train.flatten(), P_train.flatten(), 'o', label='NN Training Point')
    plt.plot(t.flatten(), P_result.flatten(), 'k', label='PINN Solution')
    plt.title('Physically Informed Neural Network Solution')
    plt.xlabel('t [s]')
    plt.ylabel('P [mmHg]')
    plt.legend()
    plt.show()

In the next cell an alternative, faster optimization for training the network which uses the SciPy compatible functionalities of Tensorflow is set up.
This method reinitializes for each run so it can be rerun immediately to yield different results.

In [ ]:
#===========================================================#
# train the model using PINNS and ScipyOptimizerInterface   #
#===========================================================#

optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, 
                                                   method = 'L-BFGS-B', 
                                                   options = {'maxiter': 500,
                                                              'maxfun': 50000,
                                                              'maxcor': 50,
                                                              'maxls': 50,
                                                              'ftol' : 1.0 * np.finfo(float).eps})
with tf.Session() as sess:
    sess.run(init)
    tf_dict = {t_tf: t_train, P_tf: P_train, 
                             t_f_tf: t_f_train, q_tf:q_train}
    optimizer.minimize(sess, 
                       feed_dict = tf_dict,         
                       fetches = [loss], 
                       loss_callback = callback)

    P_result = sess.run(P_pred, feed_dict = {t_tf: t})
    plt.figure()
    plt.plot(t.flatten(), P_data.flatten(),'b', label='Data')
    plt.plot(t_train.flatten(), P_train.flatten(), 'yo', label='NN Training Points')
    plt.plot(t.flatten(), P_result.flatten(), 'r', label='PINN Solution')
    plt.legend()
    plt.title('Physically Informed Neural Network Solution')
    plt.xlabel('P [mmHg]')
    plt.ylabel('t [s]')
    plt.show()

# PINNS applied on the steady state 1D equation for blood flow

In this notebook we will try to solve the steady state 1D momentum equation for blood flow in rigid domains. 

\begin{equation}
\frac{\partial}{\partial x } \left(\frac{Q^2}{A}\right) = -\frac{A}{\rho}\frac{\partial P}{\partial x} - \frac{8 \, \mu \, \pi \, Q}{\rho \, A}\,,
\end{equation}
which may be reformulated to

\begin{equation}
\frac{\partial P}{\partial x}  = -\frac{\rho}{A} \frac{\partial}{\partial x } \left(\frac{Q^2}{A}\right)- \frac{8 \, \mu \, \pi \, Q}{\, A^2}\,,
\end{equation}

In this case we'll treat Q as given in which and we may express P(x) as:

\begin{equation}
P\left(x\right)  = \int^x -\frac{\rho}{A} \frac{\partial}{\partial x } \left(\frac{Q^2}{A}\right) dx  + \int^x- \frac{8 \, \mu \, \pi \, Q}{\, A^2} dx\,,
\end{equation}

Our f function is defined as 
\begin{equation}
f  = \frac{\partial P}{\partial x} - I_c - I_f \,,
\end{equation}
where $I_c=-\frac{\rho}{A} \frac{\partial}{\partial x } \left(\frac{Q^2}{A}\right)$ and $I_f=- \frac{8 \, \mu \, \pi \, Q}{\, A^2}$, in which $I_c$ and $I_f$ are source terms.

## Code and functions to calculate solution and source terms for different geometries

Problem 3:
- Solve the 1D problem for different geometries; "linearTapering", "sine", "constant" using PINNS

In [ ]:
import sympy as sp
from scipy.integrate import quad

%matplotlib inline
def sinusGeom_np(x, R0, Rmin, l):
    r = R0 + ((R0 - Rmin)/2)*(np.cos(2*np.pi*x/l) - 1)
    
    return r

def sinusGeom_sp(x, R0, Rmin, l):
    r = R0 + ((R0 - Rmin)/2)*(sp.cos(2*sp.pi*x/l) - 1)
    
    return r

def linearTapering(x, R0, Rmin, l):
    
    r = R0 + (Rmin - R0)*x
    
    return r

def integrand_friction_sp(r, mu, Q):
    
    A = sp.pi*r**2
    
    I_f_sp = -8*mu*sp.pi*Q/(A**2)
    
    return I_f_sp

def integrand_convective_sp(r, rho, Q):
    
    A = sp.pi*r**2
    
    I_c_sp = -rho*sp.diff(Q**2/A)/A
    
    return I_c_sp

def calcDeltaP(x, I_f, I_c, diffusive=True, convective=True, P_in=0):
    
    P = np.zeros(len(x))
    P[0] = P_in
    P_f = np.zeros(len(x))
    P_f[0] = P_in
    P_c = np.zeros(len(x))
    P_c[0] = P_in
    I_f_array = np.zeros(len(x))
    I_c_array = np.zeros(len(x))
    if diffusive:
        I_f_array[0] = I_f(x[0])
    if convective:
        I_c_array[0] = I_c(x[0])
    for n in range(len(x) - 1):        
        
        dp = 0
        dp_f = quad(I_f, x[n], x[n + 1])[0]
        dp_c = quad(I_c, x[n], x[n + 1])[0]
        if diffusive:
            dp += dp_f
            P_f[n + 1] = P_f[n] + dp_f
            I_f_array[n + 1] = I_f(x[n + 1])
        if convective:
            dp += dp_c
            P_c[n + 1] = P_c[n] + dp_c
            I_c_array[n + 1] = I_c(x[n + 1])
        P[n + 1] = P[n] + dp
    
    return P, P_f, P_c, I_f_array, I_c_array

def get1D_data(N, geometryType="sine", showPlots=True, diffusive=True, convective=True):

    R0 = 0.2    # [cm]
    Rmin = 0.05 # [cm]
    l = 1       # [cm]
    Q = 2       # [ml/s]
    rho = 1.05  # [g/cm^3]
    mu = 0.035  # [P] (g/(cm s))
    #N = 1001
    x_np = np.linspace(0, l, N)
    
    if geometryType == "sine":
        geomFunc_np = sinusGeom_np
        geomFunc_sp = sinusGeom_sp
    elif geometryType == "linearTapering":
        geomFunc_np = linearTapering
        geomFunc_sp = linearTapering
    elif geometryType == "constant":
        geomFunc_np = linearTapering
        geomFunc_sp = linearTapering
        R0 = Rmin
    r_np = geomFunc_np(x_np, R0, Rmin, l)

    x = sp.Symbol('x')
    r_sp = geomFunc_sp(x, R0, Rmin, l)#R0 + ((R0 - Rmin)/2)*(sp.cos(2*sp.pi*x/l) - 1)

    integrand_f = integrand_friction_sp(r_sp, mu, Q)
    integrand_c = integrand_convective_sp(r_sp, rho, Q)

    integrand_f = sp.lambdify([x], integrand_f)
    integrand_c = sp.lambdify([x], integrand_c)

    P_np, P_f_np, P_c_np, integrand_f_array, integrand_c_array = calcDeltaP(x_np, integrand_f, integrand_c,
                                                                           diffusive=diffusive, convective=convective)

    #showPlots = False
    if showPlots:
        plt.figure()
        plt.plot(x_np, r_np)
        #plt.plot(x_np, linearTapering_np(x_np, R0, Rmin, l))
        plt.xlabel("x [cm]")
        plt.ylabel("r [cm]")
        plt.figure()
        plt.plot(x_np, P_np/1333.2)
        plt.plot(x_np, P_f_np/1333.2)
        plt.plot(x_np, P_c_np/1333.2)
        plt.legend(["P", "P_f", "P_c"])
        #plt.plot(x_np, linearTapering_np(x_np, R0, Rmin, l))
        plt.xlabel("x [cm]")
        plt.ylabel("P [mmHg]")

        plt.figure()
        plt.plot(x_np, integrand_f_array/1333.2)
        plt.plot(x_np, integrand_c_array/1333.2)
        plt.legend(["I_f (dP_dx_f)", "I_c (dP_dx_c)"])
        #plt.plot(x_np, linearTapering_np(x_np, R0, Rmin, l))
        plt.xlabel("x [cm]")
        plt.ylabel("dP_dx [mmHg/cm]")
    
    return x_np, r_np, P_np, P_f_np, P_c_np, integrand_f_array, integrand_c_array

#x_np, r_np, P_np, P_f_np, P_c_np, integrand_f_array, integrand_c_array = get1D_data(101)

## Code and functions do define neural net, f-function and initialization of variables

In [ ]:
#===================================================
# the remaining functions are initialized in the WK2 example
#===================================================
def net_f(x, I_f, I_c):
    
    u = net_u(x)
    
    u_x = tf.gradients(u, x)
    
    f =  u_x - I_f - I_c
    
    return f
#===================================================

## Main program

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#===================================================
# load data
#===================================================
N = 1001 # total number of x, P values
N_train = 10 # number of training points
N_train_f = 20 # number of training points
geometryType = "linearTapering" # ["sine", "linearTapering", "constant"]
layers = [1, 5, 1]
diffusive = True
convective = True
showPlots = True
x, r, P, P_f, P_c, I_f, I_c = get1D_data(N, geometryType=geometryType, showPlots=showPlots,
                                                         diffusive=diffusive, convective=convective)
dyneTommHg = 1./1333.22368

scaleInputs = True
if scaleInputs:
    P *= dyneTommHg
    P_f *= dyneTommHg
    P_c *= dyneTommHg
    I_f *= dyneTommHg
    I_c *= dyneTommHg
    
#===================================================
# turn 1D array into 2D array of shape (N, 1)
#===================================================
x = x[:, np.newaxis]
P = P[:, np.newaxis]
I_f = I_f[:, np.newaxis]
I_c = I_c[:, np.newaxis]

In [ ]:
#===================================================
# set parameters for neural network and training
#===================================================


init = tf.global_variables_initializer()

print(np.shape(I_f), np.shape(I_c))
#===================================================
# sample random points for training
#===================================================
idx = np.random.choice(x.shape[0], N_train, replace=False)
idx_f = np.random.choice(x.shape[0], N_train_f, replace=False)
x_train = x[idx]
P_train = P[idx]
#x_train = x[0:1, 0:1]
#P_train = P[0:1, 0:1]

x_train_f = x[idx_f]
I_f_train_f = np.interp(x_train_f.flatten(), x.flatten(), I_f.flatten())
I_f_train_f = I_f_train_f[:, np.newaxis]
I_c_train_f = np.interp(x_train_f.flatten(), x.flatten(), I_c.flatten())
I_c_train_f = I_c_train_f[:, np.newaxis]
print(np.shape((I_c_train_f)))
#===================================================
# set up placeholder for inputs and outputs
#===================================================
x_tf = tf.placeholder(tf.float32, shape=[None, x.shape[1]], name='x')
P_tf = tf.placeholder(tf.float32, shape=[None, P.shape[1]], name='P')

x_f_tf = tf.placeholder(tf.float32, shape=[None, x.shape[1]], name='x')
I_f_tf = tf.placeholder(tf.float32, shape=[None, I_f.shape[1]], name='integrand_f')
I_c_tf = tf.placeholder(tf.float32, shape=[None, I_c.shape[1]], name='integrand_c')
#===================================================
# initialize neural net and f functions
#===================================================
lb = x.min(0)
ub = x.max(0)
weights, biases = initialize_NN(layers)

P_pred = net_u(x_tf)
f_pred = net_f(x_f_tf, I_f_tf, I_c_tf) 

In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    P_pred_init = sess.run(P_pred, feed_dict = {x_tf:x_train})
plt.figure()
plt.plot(x.flatten(), P.flatten(), label='Data')
plt.plot(x_train.flatten(), P_train.flatten(), 'o', label='NN Training Points')
plt.plot(x_train.flatten(), P_pred_init.flatten(), label='Initialization values')
plt.legend()
plt.title('Initialization values')
plt.xlabel('x [s]')
plt.ylabel('P [mmHg]')
plt.show()

In [ ]:
#===================================================
# train the model using PINNS and GradientDescentOptimizer
#===================================================

MSE_u = tf.reduce_mean(tf.square((P_tf - P_pred)))
MSE_f = tf.reduce_mean(tf.square(f_pred))/100
loss = MSE_u + MSE_f
learning_rate_value = 0.01
learning_rate = tf.placeholder(tf.float32, shape=[])
learning_rate_value_late = 0.01#25
epochs = 10000
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        if epoch < 2000:
            learning_rate_value_epoch = learning_rate_value
        else:
            learning_rate_value_epoch = learning_rate_value_late
        _, c, MSE_u_value, MSE_f_value = sess.run([optimiser, loss, MSE_u, MSE_f], 
                     feed_dict={x_tf: x_train, P_tf: P_train, 
                                x_f_tf: x_train_f, I_f_tf: I_f_train_f,
                                I_c_tf: I_c_train_f, learning_rate: learning_rate_value_epoch})
        
        
        avg_cost += c
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost), "MSE_u =", "{:.6f}".format(MSE_u_value), "MSE_f =", "{:.6f}".format(MSE_f_value))

    P_result = sess.run(P_pred, feed_dict = {x_tf:x})
    P_result_f = sess.run(P_pred, feed_dict = {x_tf: x_train_f})
    plt.figure()
    plt.plot(x.flatten(), P.flatten())
    plt.plot(x.flatten(), P_result.flatten(), '--')
    plt.plot(x_train.flatten(), P_train.flatten(), 'o')
    plt.plot(x_train_f.flatten(), P_result_f.flatten(), 'o')
    plt.title("PINN Solution")
    plt.xlabel("x [cm]")
    plt.ylabel("P [mmHg]")
    plt.legend(["P(x)", "P_PINN", "P_train", "P_train_f"])
    plt.show()